In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import xlogy
import h5py

from sklearn.metrics import accuracy_score

In [0]:
def load_data():

  """Loads the dataset and initialize the model

    Returns
    -------
    X: array
      Input features
    y: array
      Target variables
    model: list
      List containing the layers
  """  

  # #XOR
  X = np.array([[0,0],[0,1],[1,0],[1,1]])
  y = np.array([[0],[1],[1],[0]])

  # #AND
  # X = np.array([[0,0],[0,1],[1,0],[1,1]])
  # y = np.array([[0],[0],[0],[1]])

  model = []
  model.append(Dense(X.shape[1], 3, 'relu', 1))
  model.append(Dense(3, y.shape[1], 'sigmoid', 1))

  return X, y, model

In [0]:
def relu(Z):

  """Applies relu function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying relu function
  """
  
  return np.maximum(Z, 0)

In [0]:
def relu_prime(Z):
  
  """Applies differentiation of relu function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying diff of relu function
  """

  return (Z>0).astype(Z.dtype)

In [0]:
def sigmoid(Z):

  """Applies sigmoid function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying sigmoid function
  """    
  
  return 1/(1+np.power(np.e, -Z))

In [0]:
def sigmoid_prime(Z):

  """Applies differentiation of sigmoid function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying diff of sigmoid function
  """
  
  return (1-np.power(Z, 2))

In [0]:
def leaky_relu(Z, alpha=0.01):

  """Applies leaky relu function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value
    alpha: float
      Negative slope coefficient

    Returns
    -------
    A: same shape as input
      Value after applying leaky relu function
  """   

  return np.where(Z > 0, Z, Z * alpha)

In [0]:
def leaky_relu_prime(Z, alpha=0.01):

  """Applies differentiation of leaky relu function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value
    alpha: float
      Negative slope coefficient

    Returns
    -------
    A: same shape as input
      Value after applying diff of leaky relu function
  """

  dz = np.ones_like(Z)
  dz[Z < 0] = alpha
  return dz

In [0]:
def tanh(Z):

  """Applies tanh function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying tanh function
  """   

  return np.tanh(Z)

In [0]:
def tanh_prime(Z):
  
  """Applies differentiation of tanh function to an array/value

    Arguments
    ---------
    Z: float/int/array_like
      Original Value

    Returns
    -------
    A: same shape as input
      Value after applying diff of tanh function
  """

  return 1-(tanh(Z)**2)

In [0]:
def get_activation_function(name):

  """Returns function corresponding to an activation name

    Arguments
    ---------
    name: string
      'relu', 'leaky_relu', 'tanh' or 'sigmoid' activation

    Returns
    -------
    Corresponding activation function
  """

  if name=='relu':
    return relu
  elif name=='sigmoid':
    return sigmoid
  elif name=='leaky_relu':
    return leaky_relu
  elif name=='tanh':
    return tanh
  else:
    raise ValueError('Only "relu", "leaky_relu", "tanh" and "sigmoid" supported')

In [0]:
def get_derivative_activation_function(name):

  """Returns differentiation function corresponding to an activation name

  Arguments
  ---------
  name: string
    'relu', 'leaky_relu', 'tanh' or 'sigmoid' activation

  Returns
  -------
  Corresponding diff of activation function
  """

  if name=='relu':
    return relu_prime
  elif name=='sigmoid':
    return sigmoid_prime
  elif name=='leaky_relu':
    return leaky_relu_prime
  elif name=='tanh':
    return tanh_prime
  else:
    raise ValueError('Only "relu", "leaky_relu", "tanh" and "sigmoid" supported')

In [0]:
def vector_from_model(model, params_or_grads):

  """Make a single vector of values from a model list

    Arguments
    ---------
    model: list
      List containing the layers
    params_or_grads: str
      Parameters to be converted to dictionary.
      Only 'params' or 'grads' is allowed.

    Returns
    -------
    vector: array
      Vector containing the values
    keys: list
      Keys corresponding to the values in the vector
    shapes: list
      Shapes of the parameters
    activations: list
      Activation functions of the parameters' corresponding layers
  """

  if params_or_grads=='params':
    info = ['W', 'b']
  elif params_or_grads=='grads':
    info = ['dw', 'db']
  else:
    raise ValueError('Only "params" and "grads" allowed')

  keys = []
  shapes = []
  activations = []

  for i, layer in enumerate(model):
    param_1 = getattr(layer, info[0])    # W or dw
    param_2 = getattr(layer, info[1])    # b or db
    shapes.append(param_1.shape)
    shapes.append(param_2.shape)
    vector_1 = np.array(param_1).reshape(-1, 1)
    vector_2 = np.array(param_2).reshape(-1, 1)

    if i==0:
      vector = vector_1
    else:
      vector = np.concatenate((vector, vector_1))
    
    vector = np.concatenate((vector, vector_2))
    keys.append(info[0]+str(i+1))
    keys.append(info[1]+str(i+1))
    activations.append(layer.activation)

  return vector, keys, shapes, activations

In [0]:
def model_from_vector(vector, keys, shapes, activations):

  """Make a single vector of values from a model list

    Arguments
    ---------
    vector: array
      Vector containing the values
    keys: list
      Keys corresponding to the values in the vector
    shapes: list
      Shapes of the parameters
    activations: list
      Activation functions of the parameters' corresponding layers

    Returns
    -------
    model: list
      List containing the layers
  """

  model = []
  v = vector.copy()

  count = 0

  for i in range(0, len(keys), 2):
    d = Dense(*shapes[i], activations[count])
    k = ''.join(filter(str.isalpha, keys[i]))
    setattr(d, k, v[:np.multiply(*shapes[i])].reshape(shapes[i]))
    v = v[np.multiply(*shapes[i]):]
    k = ''.join(filter(str.isalpha, keys[i+1]))
    setattr(d, k, v[:np.multiply(*shapes[i+1])].reshape(shapes[i+1]))
    v = v[np.multiply(*shapes[i+1]):]
    model.append(d)
    count += 1

  return model

In [0]:
def initialize_layer_weights(n_l_1, n_l, random_state=0):

  """Initializes random weights and bias for a layer l

    Arguments
    ---------
    n_l_1: int
      Number of neurons in previous layer (l-1)
    n_l_1: int
      Number of neurons in current layer (l)
    random_state: int
      Random seed

    Returns
    -------
    dict
      Contains the randomly initialized weights and bias arrays

      The keys for weights and bias arrays in the dict are 'W1', 'b1', 'W2' and 'b2'
  """

  np.random.seed(random_state)

  wl = np.random.randn(n_l_1, n_l) / np.sqrt(n_l_1)
  bl = np.random.randn(1, n_l) / np.sqrt(n_l_1)

  return {'wl': wl, 'bl': bl}

In [0]:
class Dense:
  
  """Returns a dense layer with randomly initialized weights and bias

    Arguments
    ---------
    input_dim: int
      Number of neurons in previous layer.
    units: int
      Number of neurons in the layer.
    activation: str
      Activation function to use. 'relu', 'leaky_relu', 'tanh' or 'sigmoid'

    Returns
    -------
    Dense layer
      An instance of the Dense layer initialized with random params.
  """

  def __init__(self, input_dim, units, activation, random_state=0):

    params = initialize_layer_weights(input_dim, units, random_state)

    self.units = units
    self.W = params['wl']
    self.b = params['bl']
    self.activation = activation
    self.Z = None
    self.A = None
    self.dz = None
    self.da = None
    self.dw = None
    self.db = None

In [0]:
def forward_prop(X, model):
  
  """Performs forward propagation and calculates output value

    Arguments
    ---------
    X: array_like
      Data
    model: list
      List containing the layers

    Returns
    -------
    Model: list
      List containing layers with updated 'Z' and 'A'
  """

  for i in range(len(model)):

    if i==0:
      X_l_1 = X.copy()
    else:
      X_l_1 = model[i-1].A

    model[i].Z = np.dot(X_l_1, model[i].W) + model[i].b
    model[i].A = get_activation_function(model[i].activation)(model[i].Z)
  
  return model

In [0]:
def calculate_loss(y, model):

  """Calculate the entropy loss

    Arguments
    --------- 
    y: array-like
      True lables
    model: list
      List containing the layers

    Returns
    -------
    loss: float
      Entropy loss
  """

  m = y.shape[0]
  A = model[-1].A

  return np.squeeze(-(1./m)*np.sum(np.multiply(np.log(A), y)+np.multiply(np.log(1-A), 1-y)))     # Squeeze will convert [[cost]] to 'cost' float variable

In [0]:
def backward_prop(X, y, model):

  """Performs backward propagation

    Arguments
    ---------
    X: array_like
      Data
    y: array_like
      True labels
    model: list
      List containing the layers

    Returns
    -------
    model: list
      List containing the layers with calculated 'dw' and 'db'
  """

  m = X.shape[0]

  for i in range(len(model)-1, -1, -1):

    if i==len(model)-1:
      model[i].dz = model[-1].A - y
      model[i].dw = 1./m * np.dot(model[i-1].A.T, model[i].dz)
      model[i].db = 1./m * np.sum(model[i].dz, axis=0, keepdims=True)

      model[i-1].da = np.dot(model[i].dz, model[i].W.T)

    else:

      model[i].dz = np.multiply(np.int64(model[i].A>0), model[i].da) * get_derivative_activation_function(model[i].activation)(model[i].Z)
      
      if i!=0:
        model[i].dw = 1./m * np.dot(model[i-1].A.T, model[i].dz)
      else:
        model[i].dw = 1./m * np.dot(X.T, model[i].dz)
      model[i].db = 1./m * np.sum(model[i].dz, axis=0, keepdims=True)
      if i!=0:
        model[i-1].da = np.dot(model[i].dz, model[i].W.T)

  return model

In [0]:
def update_weights(model, learning_rate=0.01):

  """Updates weights of the layers

    Arguments
    ---------
    model: list
      List containing the layers
    learning_rate: int, float
      Learning rate for the weight update

    Returns
    -------
    model: list
      List containing the layers
  """

  for i in range(len(model)):
    model[i].W -= learning_rate*model[i].dw
    model[i].b -= learning_rate*model[i].db
    
  return model

In [0]:
def gradient_checking(model, X, y, epsilon=1e-7):

  """Checks whether the implementation of the backpropagation is correct or not
  
  Arguments
  ---------
  model: list
    List containing the layers
  X: array_like
    Data
  y: array_like
    True Labels
  epsilon: float
    Tiny shift to the input to compute approximated gradient
    
  """

  param_vector, param_keys, param_shapes, param_activations = vector_from_model(model, 'params')
  grad_vector, grad_keys, grad_shapes, grad_activations = vector_from_model(model, 'grads')
  param_num = param_vector.shape[0]
  J_plus = np.zeros((param_num, 1))
  J_minus = np.zeros((param_num, 1))
  grad_approx = np.zeros((param_num, 1))

  for i in range(param_num):
    theta_plus = param_vector.copy()
    theta_plus[i][0] += epsilon
    J_plus[i] = calculate_loss(y, forward_prop(X, model_from_vector(theta_plus, param_keys, param_shapes, param_activations)))
    
    theta_minus = param_vector.copy()
    theta_minus[i][0] -= epsilon
    J_minus[i] = calculate_loss(y, forward_prop(X, model_from_vector(theta_minus, param_keys, param_shapes, param_activations)))
    
    grad_approx[i] = (J_plus[i]-J_minus[i]) / (2*epsilon)

  diff = np.linalg.norm(grad_vector - grad_approx) / (np.linalg.norm(grad_vector) + np.linalg.norm(grad_approx))   

  if diff>2e-7:
    print('Wrong implementation of Gradient Descent:')
  else:
    print('Correct implementation of Gradient Descent')
  print('Difference between gradient and gradient approximation:', diff)

In [22]:
X, y, model = load_data()

model = forward_prop(X, model)
model = backward_prop(X, y, model)
gradient_checking(model, X, y, epsilon=1e-7)

Correct implementation of Gradient Descent
Difference between gradient and gradient approximation: 2.0025630545815287e-09
